In [61]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import math

In [62]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean

## Now get the files with the first and the last kept duplicates

In [63]:
file_first = ['//Volumes/Elements/GitHub/twitter-project/Data Files/CRWD_first_kept_preproc.csv',
'/Volumes/Elements/GitHub/twitter-project/Data Files/APPN_first_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/INSP_first_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/SNY_first_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/EVBG_first_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/KMI_first_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/Amazon_first_kept_preproc.csv',
'/Volumes/Elements/GitHub/twitter-project/Data Files/Exxon_first_kept_preproc.csv']

In [64]:
file_last = ['/Volumes/Elements/GitHub/twitter-project/Data Files/CRWD_last_kept_preproc.csv',
'/Volumes/Elements/GitHub/twitter-project/Data Files/APPN_last_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/INSP_last_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/SNY_last_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/EVBG_last_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/KMI_last_kept_preproc.csv', 
'/Volumes/Elements/GitHub/twitter-project/Data Files/Amazon_last_kept_preproc.csv',
'/Volumes/Elements/GitHub/twitter-project/Data FilesExxon_last_kept_preproc.csv']



## Now hose with the number of days to predict ahead

In [65]:
## Shift the number of days
shift_days = 1
shift_steps = shift_days * 24  # Number of hours.

In [66]:
amzn_df = pd.read_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/Amazon_first_kept_preproc.csv', header='infer', index_col=0, parse_dates=True)
print(amzn_df.shape)
print(amzn_df.head())


(233, 14)
         date  flat_avg_neg  flat_avg_neu  flat_avg_pos  number_of_same_dates  \
0  2020-06-24      0.081976      0.454402      0.463622                 314.0   
1  2020-06-25      0.076232      0.540759      0.383009                 856.0   
2  2020-06-26      0.092678      0.559408      0.347915                 107.0   
3  2020-06-27      0.061732      0.715758      0.222509                  20.0   
4  2020-06-28      0.101550      0.460008      0.438442                  23.0   

   weighted_avg_neg  weighted_avg_neu  weighted_avg_pos   min_neg   min_neu  \
0          0.102342          0.410040          0.487618  0.000464  0.001805   
1          0.112850          0.520682          0.366468  0.000441  0.001772   
2          0.126983          0.475099          0.397917  0.000726  0.002264   
3          0.022118          0.679987          0.297890  0.001158  0.005275   
4          0.080943          0.545341          0.373712  0.000965  0.004019   

    min_pos   max_neg   max_

In [67]:
df_targets = amzn_df.shift(-shift_days)
df_targets.shape

(233, 14)

In [68]:
y_cols = amzn_df[["date","weighted_avg_neg", "weighted_avg_neu", "weighted_avg_pos"]]
y_data = y_cols.values[:-shift_days]
print(type(y_data))


<class 'numpy.ndarray'>


In [69]:
x_data = amzn_df.values[0:-shift_days]
print(type(x_data))
print("\nShape is ", x_data.shape)

<class 'numpy.ndarray'>

Shape is  (232, 14)


## Now get the data from the yahoo pricing file


In [70]:
amzn_price = pd.read_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/amzn_stock_df.csv',header='infer', index_col=0, parse_dates=True)
print(amzn_price.shape)

(481, 41)


In [71]:
prices = amzn_price[["formatted_date","pct"]]
print(prices.head())

  formatted_date       pct
0     2020-03-25 -0.027968
1     2020-03-26  0.036933
2     2020-03-27 -0.028325
3     2020-03-30  0.033603
4     2020-03-31 -0.007246


## Now let's import the stock price data

In [72]:
market_data = ['/Volumes/Elements/GitHub/twitter-project/Data Files/amzn_stock_df.csv',
               '/Volumes/Elements/GitHub/twitter-project/Data Filestsla_stock_df.csv', 
               '/Volumes/Elements/GitHub/twitter-project/Data Files/appl_stock_df.csv', 
               '/Volumes/Elements/GitHub/twitter-project/Data Files/evbg_stock_df.csv'
               '/Volumes/Elements/GitHub/twitter-project/Data Files/kmi_stock_df.csv', 
               '/Volumes/Elements/GitHub/twitter-project/Data Files/shop_stock_df.csv', 
               '/Volumes/Elements/GitHub/twitter-project/Data Files/gmed_stock_df.csv', 
               '/Volumes/Elements/GitHub/twitter-project/Data Files/sny_stock_df.csv', 
               '/Volumes/Elements/GitHub/twitter-project/Data Files/mck_stock_df.csv',
               '/Volumes/Elements/GitHub/twitter-project/Data Fileset_stock_df.csv', 
               '/Volumes/Elements/GitHub/twitter-project/Data Files/insp_stock_df.csv',
               '/Volumes/Elements/GitHub/twitter-project/Data Files/crwd_stock_df.csv',
               '/Volumes/Elements/GitHub/twitter-project/Data Files/clf_stock_df.csv', 
               '/Volumes/Elements/GitHub/twitter-project/Data Files/UVXY_etf_df.csv', 
               '//Volumes/Elements/GitHub/twitter-project/Data FilesGLD_etf_df.csv', 
               '/Volumes/Elements/GitHub/twitter-project/Data Files/GBTC_etf_df.csv']

## Deal with the missing days of the week - in stock data

In [73]:
new_amzn_dates = pd.DataFrame()
##print("\n number of twitter dates is ", amzn_df.shape)
##print("\n number of stock dates is ", amzn_price.shape)
##print("\n head of twitter dates is ", amzn_df.head())
##print("\n head of stock dates is ", amzn_price.head())
##print("\n tale of twitter dates is ", amzn_df.tail())
##print("\n tale of stock dates is ", amzn_price.tail())

county = 0
ones = 0
ones_twos = 0
twos_ones = 0
two_two = 0
one_zed = 0
zed_one = 0
if amzn_df.shape[0] <= amzn_price.shape[0]:
    bobby = amzn_df.shape[0] - 2
else:
    bobby = amzn_price.shape[0] - 2
for datez in range(bobby):
    if amzn_price.loc[datez,"formatted_date"] == amzn_df.loc[datez,"date"]:
        new_amzn_dates.loc[datez,"date"] = amzn_df.loc[datez,"date"]
        new_amzn_dates.loc[datez,"pct"] = amzn_price.loc[datez,"pct"]
        county += 1
##        print("date is ", amzn_df.loc[datez,"date"], "and the index number is ", datez)
    else:
        new_amzn_dates.loc[datez, "date"] = amzn_df.loc[datez,"date"]
        if any (amzn_price.formatted_date == amzn_df.loc[datez,"date"]):
            
            new_amzn_dates.loc[datez, "pct"] = amzn_price.loc[amzn_price.index[amzn_price['formatted_date'] ==  amzn_df.loc[datez,"date"]].tolist()[0], "pct"]
        elif any(amzn_price.formatted_date == amzn_df.loc[datez-1,"date"]) and any(amzn_price.formatted_date == amzn_df.loc[datez+1,"date"]):  
##            print("one and one")
            ones += 1
##            new_amzn_dates.loc[datez, "pct"] = amzn_price.loc[amzn_price.index[amzn_price['formatted_date'] ==  amzn_df.loc[datez,"date"]].tolist()[0], "pct"]
        elif any(amzn_price.formatted_date == amzn_df.loc[datez-1,"date"]) and any(amzn_price.formatted_date == amzn_df.loc[datez+2,"date"]):
##            print("one and two")
            ones_twos += 1
        elif any(amzn_price.formatted_date == amzn_df.loc[datez-2,"date"]) and any(amzn_price.formatted_date == amzn_df.loc[datez+1,"date"]):
##            print("two and one")
            twos_ones += 1
        elif any(amzn_price.formatted_date == amzn_df.loc[datez-2,"date"]) and any(amzn_price.formatted_date == amzn_df.loc[datez+2,"date"]):
##            print("two and two")
            two_two += 1
        elif any(amzn_price.formatted_date == amzn_df.loc[datez-1,"date"]) and any(amzn_price.formatted_date == amzn_df.loc[datez+3,"date"]): 
            one_zed += 1
        ##elif any(amzn_price.formatted_date == amzn_df.loc[datez+3,"date"]) and any(amzn_price.formatted_date == amzn_df.loc[datez-1,"date"]): 
          ##  ozed_one += 1   
            
print("\n the count is ", county)
print("\n the one zed is ", one_zed)
print("\n the zed one is ", zed_one)
print("\n the ones is ", ones)
print("\n the onestwos is ", ones_twos)
print("\n the two ones is ", twos_ones)
print("\n the two two is ", two_two)


print("\n the head for the new data frame is \n ", new_amzn_dates.head())
print("\n the tail for the new data frame is \n ", new_amzn_dates.tail(10))


    


 the count is  0

 the one zed is  0

 the zed one is  0

 the ones is  16

 the onestwos is  17

 the two ones is  17

 the two two is  0

 the head for the new data frame is 
           date       pct
0  2020-06-24 -0.010856
1  2020-06-25  0.007380
2  2020-06-26 -0.022403
3  2020-06-27       NaN
4  2020-06-28       NaN

 the tail for the new data frame is 
             date       pct
221  2021-08-16  0.001524
222  2021-08-17 -0.017287
223  2021-08-18 -0.012566
224  2021-08-20  0.003827
225  2021-08-21       NaN
226  2021-08-22       NaN
227  2021-08-23  0.020600
228  2021-08-28       NaN
229  2021-08-29       NaN
230  2021-08-30  0.021477


In [74]:
print(amzn_df.loc[amzn_df['date']== '2020-06-29'])
print(amzn_price.loc[amzn_price['formatted_date']== '2020-06-29'])

         date  flat_avg_neg  flat_avg_neu  flat_avg_pos  number_of_same_dates  \
5  2020-06-29      0.049641      0.547178      0.403181                 112.0   

   weighted_avg_neg  weighted_avg_neu  weighted_avg_pos   min_neg  min_neu  \
5          0.020339           0.43734           0.54232  0.000528  0.00229   

    min_pos  max_neg   max_neu   max_pos  
5  0.002316  0.98605  0.990433  0.996592  
           high          low        open        close   volume     adjclose  \
66  2696.800049  2630.080078  2690.01001  2680.379883  4223400  2680.379883   

   formatted_date       pct  rolling3  var3  ...  rolling50  varfifty  \
66     2020-06-29 -0.004638 -0.006554     0  ...     0.0026         0   

    covarfifty  betafifty  rolling250  varyear  covaryear  betayr_clf  \
66           0          0    0.005076        0          0           0   

    betayr_ndaq  betayr_djia  
66            0            0  

[1 rows x 41 columns]


In [75]:
new_amzn_dates = pd.DataFrame()
##print("\n number of twitter dates is ", amzn_df.shape)
##print("\n number of stock dates is ", amzn_price.shape)
##print("\n head of twitter dates is ", amzn_df.head())
##print("\n head of stock dates is ", amzn_price.head())
##print("\n tale of twitter dates is ", amzn_df.tail())
##print("\n tale of stock dates is ", amzn_price.tail())

county = 0
ones = 0
ones_twos = 0
twos_ones = 0
two_two = 0
one_zed = 0
zed_one = 0

for datez in amzn_df.date:
    if any (amzn_price.formatted_date == datez):
        idx = amzn_df.index()____ .tolist()
        

SyntaxError: invalid syntax (9711008.py, line 19)

In [ ]:
print(amzn_df.head())